# Pokémon Analysis
![](http://cdn5.applesencia.com/wp-content/blogs.dir/17/files/2016/07/pokemon.jpg)

In this Notebook, we'll do some analysis using the data of the 721 Pokémon available until the 6th generation, including alternative forms and megas. In order to do this, we'll use the dataset available for free on [Kaggle](https://www.kaggle.com/abcsds/pokemon). 

## Introduction
We'll start by loading all the Pokémon information (currently stored in a CSV file) into a Pandas DataFrame. Besides, we'll also change the names of the columns which contain spaces in order to remove them so we can easily access the information.

In [1]:
import pandas as pd

filename = 'Pokemon.csv'
PokeData = pd.read_csv(filename, index_col = "#")

# Removing spaces from the names of the columns
cols = list(PokeData.columns)
cols[1:3] = ['Type1', 'Type2']
cols[7:9] = ['SpAtk', 'SpDef']
PokeData.columns = cols


Now, we'll start with the analysis!

## Plotting - Type comparison

The first thing we'll do will be to plot their data so we can visualize it easily. We'll start with a scatter plot in which we'll represent every Pokémon according to its stats. However, as there are a lot of possible stats (6 in total), we'll plot 3 different graphs: one with Attacks - Defenses, one with non-Special stats - Special stats, one with Attacks - Speed (good for detecting Fast Attackers) and Defense - HP (good for detecting Tough Defenders). 

Besides, we'll plot each type separately, so we can see how the type of a Pokémon correlates with its stats. 
We'll plot them in the following order: Bug, Dark, Dragon, Electric, Fairy, Flying, Fighting, Fire, Ghost, Grass, Ground, Ice, Normal, Poison, Psychic, Rock, Steel, Water


### Adding stats
We'll add the combined stats we're going to need to represent the data to the DataFrame. By doing this now instead of when plotting the data, we'll avoid computing the values twice for dual-typed Pokémon



In [2]:
new_stats = pd.DataFrame({'Attacks': PokeData.Attack + PokeData.SpAtk, 
                       'Defenses': PokeData.Defense + PokeData.SpDef,
                       'Specials': PokeData.SpAtk + PokeData.SpDef,
                       'Non_specials': PokeData.Attack + PokeData.Defense})

PokeData = pd.concat([PokeData, new_stats], axis = 1)

### Settings
We'll start by importing all the libraries we're going to use during this plotting section.

The main plotting library will be [Bokeh](http://bokeh.pydata.org/en/latest/), as with it we can interact with the plot. We'll use this feature in order to show a small label with the Pokémon name and its data when we hover it. 

We will also import some standard libraries in order to perform the needed operations easily

In [3]:
from collections import OrderedDict
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import gridplot
from bokeh.charts import Scatter
from bokeh.models import ColumnDataSource, HoverTool, Range1d


### Plot function

Now, we'll define a generic plot function that will get a list of Pokémon as a parameter and plot their data.

In [4]:
def plot_stats(PokeData, currentType, currentTypeColor):
    
    # Selecting the Pokémon with the current type
    pokes = PokeData.loc[(PokeData.Type1 == currentType) | (PokeData.Type2 == currentType)]

    #Plotting part 
    TOOLS = "hover,box_select,pan,wheel_zoom,box_zoom,undo,redo,reset,tap,save"

    size = 480
    
    # Creating the plotting elements
    sharedx_range = Range1d(0,510)
    sharedy_range = Range1d(0,510)

    #Preparing data source
    source = ColumnDataSource(pokes)
    
    #PLOT 1: ATTACKS VS DEFENSES
    f1 = figure(tools = TOOLS, width = size, height = size)
    f1.circle(source = source, x = "Attacks", y = "Defenses", radius=4,
             line_width = 1, line_alpha = 1, line_color = "black", 
              color = currentTypeColor)
    # Setting tools
    hover = f1.select(dict(type=HoverTool))
    hover.tooltips = OrderedDict([
        ("Pokémon", "@Name"),
        ("HP", "@HP"),
        ("Attack", "@Attack"),
        ("Defense", "@Defense"),
        ("Sp. Attack", "@SpAtk"),
        ("Sp. Defense", "@SpDef"),
        ("Speed", "@Speed"),
    ])
    #By setting a common range, we'll link the plots
    f1.x_range = sharedx_range
    f1.y_range = sharedy_range
    
    
    #PLOT 2: SPECIALS VS NON-SPECIALS
    f2 = figure(tools = TOOLS,  width = size, height = size)
    f2.circle(source = source, x = "Specials", y = "Non_specials", radius=4,
             line_width = 1, line_alpha = 1, line_color = "black", 
              color = currentTypeColor)
    # Setting tools
    hover = f2.select(dict(type=HoverTool))
    hover.tooltips = OrderedDict([
        ("Pokémon", "@Name"),
        ("HP", "@HP"),
        ("Attack", "@Attack"),
        ("Defense", "@Defense"),
        ("Sp. Attack", "@SpAtk"),
        ("Sp. Defense", "@SpDef"),
        ("Speed", "@Speed"),
    ])
    #By setting a common range, we'll link the plots
    f2.x_range = sharedx_range
    f2.y_range = sharedy_range
    
    
    #PLOT 3: ATTACKS VS SPEED
    f3 = figure(tools = TOOLS,  width = size, height = size)
    f3.circle(source = source, x = "Attacks", y = "Speed", radius=4,
             line_width = 1, line_alpha = 1, line_color = "black", 
              color = currentTypeColor)
    # Setting tools
    hover = f3.select(dict(type=HoverTool))
    hover.tooltips = OrderedDict([
        ("Pokémon", "@Name"),
        ("HP", "@HP"),
        ("Attack", "@Attack"),
        ("Defense", "@Defense"),
        ("Sp. Attack", "@SpAtk"),
        ("Sp. Defense", "@SpDef"),
        ("Speed", "@Speed"),
    ])
    #By setting a common range, we'll link the plots
    f3.x_range = sharedx_range
    f3.y_range = Range1d(0,255)
    
    
    
    #PLOT 4: DEFENSES VS HP
    f4 = figure(tools = TOOLS,  width = size, height = size)
    f4.circle(source = source, x = "Defenses", y = "HP", radius=4,
             line_width = 1, line_alpha = 1, line_color = "black", 
              color = currentTypeColor)
    # Setting tools
    hover = f4.select(dict(type=HoverTool))
    hover.tooltips = OrderedDict([
        ("Pokémon", "@Name"),
        ("HP", "@HP"),
        ("Attack", "@Attack"),
        ("Defense", "@Defense"),
        ("Sp. Attack", "@SpAtk"),
        ("Sp. Defense", "@SpDef"),
        ("Speed", "@Speed"),
    ])
    #By setting a common range, we'll link the plots
    f4.x_range = sharedx_range
    f4.y_range = Range1d(0,255)
    
    
    f = gridplot([[f1, f2], [f3, f4]])
    output_notebook()
    show(f)

### Bug

We'll start with the bug type Pokémon


In [5]:
#Processing data source
currentType = "Bug"
currentTypeColor = "olive"

plot_stats(PokeData, currentType, currentTypeColor)

Loading BokehJS ...

### Dark
Now is time of plotting the Dark type Pokémon. We can reuse most of the code we've already used.

In [6]:
currentType = "Dark"
currentTypeColor = "Black"

plot_stats(PokeData, currentType, currentTypeColor)

Loading BokehJS ...

### Dragon
Now it's Dragon time. Someone call the Targaryen! /joke

In [7]:
currentType = "Dragon"
currentTypeColor = "DarkBlue"

plot_stats(PokeData, currentType, currentTypeColor)

Loading BokehJS ...

### Electric
Let's see now how the Electrics are represented


In [24]:
currentType = "Electric"
currentTypeColor = "Yellow"

plot_stats(PokeData, currentType, currentTypeColor)

Loading BokehJS ...

### Fairy
Now is the turn of the newest incorporation, which will probably make them smaller than the other types

In [9]:
currentType = "Fairy"
currentTypeColor = "Pink"

plot_stats(PokeData, currentType, currentTypeColor)

Loading BokehJS ...

### Flying
Let's see how high they can get!

In [10]:
currentType = "Flying"
currentTypeColor = "LightBlue"

plot_stats(PokeData, currentType, currentTypeColor)

Loading BokehJS ...

### Fighting
And now it's time to see those ~~Fire starters~~ tough fighters

In [11]:
currentType = "Fighting"
currentTypeColor = "Brown"

plot_stats(PokeData, currentType, currentTypeColor)

Loading BokehJS ...

### Fire
Time to burn them all!

In [12]:
currentType = "Fire"
currentTypeColor = "Red"

plot_stats(PokeData, currentType, currentTypeColor)

Loading BokehJS ...

### Ghost
It's important to say that you shouldn't expect to see Patrick Swayze or Demi Moore here. Don't say I didn't tell you. 

In [13]:
currentType = "Ghost"
currentTypeColor = "DarkMagenta"

plot_stats(PokeData, currentType, currentTypeColor)

Loading BokehJS ...

### Grass
Here I'd make a joke about vegetarians, but we're in 2016 and ~~everyone will be offended~~ we know that's not something we can joke about.

In [14]:
currentType = "Grass"
currentTypeColor = "Green"

plot_stats(PokeData, currentType, currentTypeColor)

Loading BokehJS ...

### Ground
Time to Earthquake everyone!

In [15]:
currentType = "Ground"
currentTypeColor = "Brown"

plot_stats(PokeData, currentType, currentTypeColor)

Loading BokehJS ...

### Ice
Time to cool down things

In [16]:
currentType = "Ice"
currentTypeColor = "LightBlue"

plot_stats(PokeData, currentType, currentTypeColor)

Loading BokehJS ...

### Normal
I won't make jokes here. They've had enough by being Normal-typed

In [17]:
currentType = "Normal"
currentTypeColor = "Orange"

plot_stats(PokeData, currentType, currentTypeColor)

Loading BokehJS ...

### Poison
Prepare those antidotes!

In [18]:
currentType = "Poison"
currentTypeColor = "Purple"

plot_stats(PokeData, currentType, currentTypeColor)

Loading BokehJS ...

### Psychic
Are you ready to bend those spoons?

In [23]:
currentType = "Psychic"
currentTypeColor = "Violet"

plot_stats(PokeData, currentType, currentTypeColor)

Loading BokehJS ...

### Rock
I'm so excited too see how high it's defense will be that I don't have time to think a joke


In [20]:
currentType = "Rock"
currentTypeColor = "Grey"

plot_stats(PokeData, currentType, currentTypeColor)

Loading BokehJS ...

### Steel
The other tough guys of the game. Will they really be harder than the rocks?

In [21]:
currentType = "Steel"
currentTypeColor = "Grey"

plot_stats(PokeData, currentType, currentTypeColor)

Loading BokehJS ...

### Water
Did you remember to bring your swimsuit?

In [22]:
currentType = "Water"
currentTypeColor = "Blue"

plot_stats(PokeData, currentType, currentTypeColor)

Loading BokehJS ...